In [1]:
pip install transformers datasets evaluate

Note: you may need to restart the kernel to use updated packages.


In [6]:
from huggingface_hub import notebook_login

In [8]:
notebook_login()

In [9]:
from datasets import load_dataset

In [16]:
squad = load_dataset("squad" , split= "train[:5000]")

In [17]:
squad = squad.train_test_split(test_size= 0.2)

In [20]:
squad["train"][20] 

{'id': '56d133d017492d1400aabc22',
 'title': 'The_Legend_of_Zelda:_Twilight_Princess',
 'context': 'Special bundles of the game contain a Wolf Link Amiibo figurine, which unlocks a Wii U-exclusive dungeon called the "Cave of Shadows" and can carry data over to the upcoming 2016 Zelda game. Other Zelda-related Amiibo figurines have distinct functions: Link and Toon Link replenish arrows, Zelda and Sheik restore Link\'s health, and Ganondorf causes Link to take twice as much damage.',
 'question': 'What characters will be able to replenish arrows?',
 'answers': {'text': ['Link and Toon'], 'answer_start': [253]}}

In [21]:
squad["test"][10]

{'id': '56d4cee32ccc5a1400d8324a',
 'title': 'Beyoncé',
 'context': 'On February 6, 2016, one day before her performance at the Super Bowl, Beyoncé released a new single exclusively on music streaming service Tidal called "Formation".',
 'question': 'When did Beyoncé release Formation?',
 'answers': {'text': ['February 6, 2016'], 'answer_start': [3]}}

In [22]:
squad['train'].column_names


['id', 'title', 'context', 'question', 'answers']

In [23]:
import huggingface_hub
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")


In [24]:
squad['train'].column_names

['id', 'title', 'context', 'question', 'answers']

In [27]:
>>> def preprocess_function(examples):
...     questions = [q.strip() for q in examples["question"]]
...     inputs = tokenizer(
...         questions,
...         examples["context"],
...         max_length=384,
...         truncation="only_second",
...         return_offsets_mapping=True,
...         padding="max_length",
...     )

...     offset_mapping = inputs.pop("offset_mapping")
...     answers = examples["answers"]
...     start_positions = []
...     end_positions = []

...     for i, offset in enumerate(offset_mapping):
...         answer = answers[i]
...         start_char = answer["answer_start"][0]
...         end_char = answer["answer_start"][0] + len(answer["text"][0])
...         sequence_ids = inputs.sequence_ids(i)

...         # Find the start and end of the context
...         idx = 0
...         while sequence_ids[idx] != 1:
...             idx += 1
...         context_start = idx
...         while sequence_ids[idx] == 1:
...             idx += 1
...         context_end = idx - 1

...         # If the answer is not fully inside the context, label it (0, 0)
...         if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
...             start_positions.append(0)
...             end_positions.append(0)
...         else:
...             # Otherwise it's the start and end token positions
...             idx = context_start
...             while idx <= context_end and offset[idx][0] <= start_char:
...                 idx += 1
...             start_positions.append(idx - 1)

...             idx = context_end
...             while idx >= context_start and offset[idx][1] >= end_char:
...                 idx -= 1
...             end_positions.append(idx + 1)

...     inputs["start_positions"] = start_positions
...     inputs["end_positions"] = end_positions
...     return inputs

In [36]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [37]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
pip install accelerate

In [47]:

>>> from transformers import DefaultDataCollator

>>> data_collator = DefaultDataCollator()

In [48]:
>>> from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer

>>> model = AutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [49]:
training_args = TrainingArguments(
    output_dir="my_qa_model",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True
)
trainer =Trainer(
    model=model,
    args = training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    processing_class=tokenizer,
    data_collator= data_collator ,
    

)
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.410628
2,2.789800,1.771497
3,2.789800,1.644778


TrainOutput(global_step=750, training_loss=2.3578301595052085, metrics={'train_runtime': 12107.7907, 'train_samples_per_second': 0.991, 'train_steps_per_second': 0.062, 'total_flos': 1175877900288000.0, 'train_loss': 2.3578301595052085, 'epoch': 3.0})

In [50]:
trainer.push_to_hub()

events.out.tfevents.1745367033.SHASHHEG-5CHDHW3.17268.2:   0%|          | 0.00/6.33k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/shashankheg/my_qa_model/commit/e6de9130e98df2b02e7cb173b56abec9b28ac310', commit_message='End of training', commit_description='', oid='e6de9130e98df2b02e7cb173b56abec9b28ac310', pr_url=None, repo_url=RepoUrl('https://huggingface.co/shashankheg/my_qa_model', endpoint='https://huggingface.co', repo_type='model', repo_id='shashankheg/my_qa_model'), pr_revision=None, pr_num=None)

In [51]:
#Optimize function to finetune the tensorflow model
from sched import scheduler
from transformers import create_optimizer
batch_size =16
num_epochs = 2
total_train_steps =((len(tokenized_squad['train']))/batch_size)*num_epochs
optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=total_train_steps,
    
)

In [57]:
from huggingface_hub import notebook_login
notebook_login()

In [60]:
>>> from transformers import TFAutoModelForQuestionAnswering

>>> model = TFAutoModelForQuestionAnswering.from_pretrained("distilbert/distilbert-base-uncased")


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForQuestionAnswering: ['vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForQuestionAnswering from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it

In [61]:
#Convert your datasets to the tf.data.Dataset format with [~transformers.TFPreTrainedModel.prepare_tf_dataset]:
>>> tf_train_set = model.prepare_tf_dataset(
...     tokenized_squad["train"],
...     shuffle=True,
...     batch_size=16,
...     collate_fn=data_collator,
... )

>>> tf_validation_set = model.prepare_tf_dataset(
...     tokenized_squad["test"],
...     shuffle=False,
...     batch_size=16,
...     collate_fn=data_collator,
... )

In [62]:
import tensorflow as tf
model.compile(optimizer=optimizer)

In [70]:
>>> from transformers.keras_callbacks import PushToHubCallback

>>> callback = PushToHubCallback(
...     output_dir="/my_qa_model",
...     tokenizer=tokenizer,
... )

Cloning https://huggingface.co/shashankheg/my_qa_model into local empty directory.


Download file model.safetensors:   0%|          | 14.7k/253M [00:00<?, ?B/s]

Download file runs/Apr22_18-06-09_SHASHHEG-5CHDHW3/events.out.tfevents.1745366771.SHASHHEG-5CHDHW3.17268.0: 10…

Download file runs/Apr22_18-10-33_SHASHHEG-5CHDHW3/events.out.tfevents.1745367033.SHASHHEG-5CHDHW3.17268.2: 10…

Download file training_args.bin: 100%|##########| 5.18k/5.18k [00:00<?, ?B/s]

Download file runs/Apr22_18-09-24_SHASHHEG-5CHDHW3/events.out.tfevents.1745366965.SHASHHEG-5CHDHW3.17268.1: 10…

Clean file runs/Apr22_18-06-09_SHASHHEG-5CHDHW3/events.out.tfevents.1745366771.SHASHHEG-5CHDHW3.17268.0: 100%|…

Clean file runs/Apr22_18-10-33_SHASHHEG-5CHDHW3/events.out.tfevents.1745367033.SHASHHEG-5CHDHW3.17268.2: 100%|…

Clean file training_args.bin: 100%|##########| 5.18k/5.18k [00:00<?, ?B/s]

Clean file runs/Apr22_18-09-24_SHASHHEG-5CHDHW3/events.out.tfevents.1745366965.SHASHHEG-5CHDHW3.17268.1: 100%|…

In [76]:
>>> model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3)

In [90]:
>>> question = 'When did Beyoncé release Formation?'
>>> context = 'On February 6, 2016, one day before her performance at the Super Bowl, Beyoncé released a new single exclusively on music streaming service Tidal called "Formation"'

In [91]:
>>> from transformers import pipeline

>>> question_answerer = pipeline("question-answering", model="my_qa_model")
>>> question_answerer(question=question, context=context)


Device set to use cpu


{'score': 0.3482300341129303,
 'start': 3,
 'end': 19,
 'answer': 'February 6, 2016'}

In [79]:
>>> from transformers import AutoTokenizer

>>> tokenizer = AutoTokenizer.from_pretrained("my_qa_model")
>>> inputs = tokenizer(question, context, return_tensors="pt")

In [82]:
>>> import torch
>>> from transformers import AutoModelForQuestionAnswering

>>> model = AutoModelForQuestionAnswering.from_pretrained("my_qa_model")
>>> with torch.no_grad():
...     outputs = model(**inputs)
outputs

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-2.7567, -0.9099, -0.9439, -3.1180, -4.5906, -3.8437, -3.1428, -4.4424,
         -4.6074, -4.1999, -2.6713, -2.7966,  4.8902,  0.3930, -2.1254, -3.8282,
         -2.3827, -2.0333,  1.2862, -3.1707,  5.4366, -2.3450,  3.0304, -3.9342,
         -2.1986,  5.0488, -3.2159, -4.2492, -3.9939, -4.1999]]), end_logits=tensor([[-3.0765, -2.2695, -0.7537, -3.7772, -1.5518, -4.0324, -2.5278, -3.8324,
         -4.6308, -1.2707, -2.8741, -3.9547,  0.7762,  3.4770,  0.8205, -3.9714,
         -4.3026, -3.2251,  0.1135, -4.1172,  3.8016,  1.0495, -0.1765,  0.5171,
         -3.9315,  5.0966, -2.7018,  0.8294, -0.7932, -1.2704]]), hidden_states=None, attentions=None)

In [83]:
>>> answer_start_index = outputs.start_logits.argmax()
>>> answer_end_index = outputs.end_logits.argmax()

In [84]:
>>> predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
>>> tokenizer.decode(predict_answer_tokens)
'176 billion parameters and can generate text in 46 languages natural languages and 13'

'176 billion parameters and can generate text in 46 languages natural languages and 13'

In [85]:
>>> from transformers import AutoTokenizer

>>> tokenizer = AutoTokenizer.from_pretrained("my_qa_model")
>>> inputs = tokenizer(question, context, return_tensors="tf")

In [86]:
>>> from transformers import TFAutoModelForQuestionAnswering

>>> model = TFAutoModelForQuestionAnswering.from_pretrained("my_qa_model")
>>> outputs = model(**inputs)

All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [87]:
>>> answer_start_index = int(tf.math.argmax(outputs.start_logits, axis=-1)[0])
>>> answer_end_index = int(tf.math.argmax(outputs.end_logits, axis=-1)[0])

In [88]:
>>> predict_answer_tokens = inputs.input_ids[0, answer_start_index : answer_end_index + 1]
>>> tokenizer.decode(predict_answer_tokens)
'176 billion parameters and can generate text in 46 languages natural languages and 13'

'176 billion parameters and can generate text in 46 languages natural languages and 13'

In [89]:
'context': 'On February 6, 2016, one day before her performance at the Super Bowl, Beyoncé released a new single exclusively on music streaming service Tidal called "Formation".',
 'question': 'When did Beyoncé release Formation?'

SyntaxError: illegal target for annotation (3793779711.py, line 1)

In [107]:
question: 'What characters will be able to replenish arrows?'
context: 'Special bundles of the game contain a Wolf Link Amiibo figurine, which unlocks a Wii U-exclusive dungeon called the "Cave of Shadows" and can carry data over to the upcoming 2016 Zelda game. Other Zelda-related Amiibo figurines have distinct functions: Link and Toon Link replenish arrows, Zelda and Sheik restore Link\'s health, and Ganondorf causes Link to take twice as much damage.'


In [110]:
>>> from transformers import pipeline

>>> question_answerer = pipeline("question-answering", model="my_qa_model")
result = question_answerer(question=question, context=context)
print(result)

Device set to use cpu
